In [5]:
import numpy
import scipy.special
import matplotlib.pyplot
%matplotlib inline
# определение класса нейронной сети
class neuralNetwork:
    
    # инициализировать нейронную сеть
    def __init__(self,inputnodes,hiddennodes,outputnodes,learningrate):
        
        self.inodes = inputnodes #кол-во узлов во входном слое
        self.hnodes = hiddennodes #кол-во узлов в крытом слое
        self.onodes = outputnodes #кол-во узлов в выходном слое
        
        # 1) центр нормального распределения установлен здесь в 0,0
        # 2) Стандартное отклонение вычисляется по количеству узлов в следующем слое с помощью функции pow(self .hnodes, -0.5),
        #    которая просто возводит количество узлов в степень -0,5.
        # 3) Конфигурация массива
        # матрица весовых коэффициентов между входным и скрытым слоями
        self.wih = numpy.random.normal(0.0, pow(self.hnodes, -0.5),(self.hnodes,self.inodes))
        # матрица весовых коэффициентов между скрытым и выходным слоями
        self.who = numpy.random.normal(0.0, pow(self.onodes, -0.5),(self.onodes,self.hnodes))
        
        
        self.lr = learningrate # коэффициент обучения
        
        # использование сигмоиды в качестве функции активации
        # expit() - это сигмода y=1/(1+e^-x)
        # лямюда-выражение(анонимная функция)
        self.activation_function = lambda x: scipy.special.expit(x)
        
        pass
    
    # тренировка нейронной сети
    def train(self, inputs_list, targets_list):
       
        # преобразовать список входных значений в двухмерный массив
        inputs = numpy.array(inputs_list, ndmin=2).T #транспонированая
        # тренировочные примеры
        targets = numpy.array(targets_list,ndmin=2).T
        
        # рассчитать входящие сигналы для скрытого слоя 
        hidden_inputs = numpy.dot(self.wih, inputs)
        # рассчитать исходящие сигналы для скрытого слоя 
        hidden_outputs = self.activation_function(hidden_inputs)
        # рассчитать входящие сигналы для выходного слоя 
        final_inputs = numpy.dot(self.who, hidden_outputs)
        # рассчитать исходящие сигналы для выходного слоя 
        final_outputs = self.activation_function(final_inputs)
        
        #ошибка = целевое значение - фактическое значение
        output_errors = targets - final_outputs
        
        #обратное распространение ошибки для узлов скрытого слоя
        #ошибки(скрытый)= веса(скрытый_выходной)^Т * ошибки(выходной)
        # распределенные пропорционально весовым коэффициентам связей и рекомбинированные на скрытых узлах
        hidden_errors = numpy.dot(self.who.T, output_errors)
        
        # обновить весовые коэффициенты связей между скрытым и выходным слоями
        self.who += self.lr*numpy.dot((output_errors*final_outputs*(1.0 - final_outputs)),numpy.transpose(hidden_outputs))
        # обновить весовые коэффициенты связей между входным и скрытым слоями
        self.wih += self.lr*numpy.dot((hidden_errors*hidden_outputs*(1.0 - hidden_outputs)),numpy.transpose(inputs))
        pass
    
    # опрос нейронной сети
    def query(self,inputs_list): #входные данные 
      
        # преобразовать список входных значений в двухмерный массив
        inputs = numpy.array(inputs_list, ndmin = 2).T
        
        # рассчитать входящие сигналы для скрытого слоя 
        # X(h) = W(ih) * I
        hidden_inputs = numpy.dot(self.wih,inputs)
        # рассчитать исходящие сигналы для скрытого слоя
        # O(h) = sigmoida(X(h))
        hidden_outputs = self.activation_function(hidden_inputs)
        
        # рассчитать входящие сигналы для выходного слоя
        # X(o) = X(ho) * O(h)
        final_inputs = numpy.dot(self.who,hidden_outputs)
        # рассчитать исходящие сигналы для выходного слоя
        # O(o) = sigmoida(X(o)
        final_outputs = self.activation_function(final_inputs)
        
        return final_outputs #выходные данные 

In [6]:
# количество входных, скрытых и выходных узлов
input_nodes = 784
hidden_nodes = 200
output_nodes = 10
# коэффициент обучения равен 0,3
learningrate = 0.1
n = neuralNetwork(input_nodes,hidden_nodes,output_nodes, learningrate)

In [7]:
training_data_file = open("mnist_dataset/mnist_train_100.csv", 'r')
training_data_list = training_data_file.readlines()
training_data_file.close()

In [8]:
# тренировка нейронной сети

# переменная epochs указывает, сколько раз тренировочный
# набор данных используется для тренировки сети
epochs = 2

for e in range(epochs):
    # перебрать все записи в тренировочном наборе данных
    for record in training_data_list:
        # получить список значений из записи, используя символы запятой (', 1) в качестве разделителей
        all_values = record.split(',')
        # масштабировать и сместить входные
        inputs = (numpy.asfarray(all_values[1:]) / 255.0 * 0.99) + 0.01
        # создать целевые выходные значения (все равны 0,01, за исключением желаемого маркерного значения, равного 0,99)
        targets = numpy.zeros(output_nodes) + 0.01
        # all_values[0] - целевое маркерное значение для данной записи
        targets[int(all_values[0])] = 0.99
        n.train(inputs, targets)
        pass
    pass

In [9]:
# тестирование нейронной сети
# журнал оценок работы сети, первоначально пустой 
scorecard = []
# перебрать все записи в тестовом наборе данных 
for record in training_data_list:
    # получить список значений из записи, используя символы
    # запятой (*,1) в качестве разделителей 
    all_values = record.split(',')
    # правильный ответ - первое значение 
    correct_label = int(all_values[0]) 
    print(correct_label, "истинный маркер")
    # масштабировать и сместить входные значения
    inputs = (numpy.asfarray(all_values[1:]) / 255.0 * 0.99) + 0.01
    # опрос сети
    outputs = n.query(inputs)
    # индекс наибольшего значения является маркерным значением 
    label = numpy.argmax(outputs)
    print(label, "ответ сети")
    # присоединить оценку ответа сети к концу списка 
    if (label == correct_label) :
        # в случае правильного ответа сети присоединить
        # к списку значение 1 
        scorecard.append(1)
    else:
        # в случае неправильного ответа сети присоединить
        # к списку значение 0 
        scorecard.append(0) 
        pass
    pass

print(scorecard)
# рассчитать показатель эффективности в виде
# доли правильных ответов 
scorecard_array = numpy.asarray(scorecard)
print ("эффективность = ", scorecard_array.sum() / scorecard_array.size)


5 истинный маркер
3 ответ сети
0 истинный маркер
0 ответ сети
4 истинный маркер
4 ответ сети
1 истинный маркер
1 ответ сети
9 истинный маркер
9 ответ сети
2 истинный маркер
2 ответ сети
1 истинный маркер
1 ответ сети
3 истинный маркер
3 ответ сети
1 истинный маркер
1 ответ сети
4 истинный маркер
4 ответ сети
3 истинный маркер
3 ответ сети
5 истинный маркер
5 ответ сети
3 истинный маркер
3 ответ сети
6 истинный маркер
6 ответ сети
1 истинный маркер
1 ответ сети
7 истинный маркер
7 ответ сети
2 истинный маркер
2 ответ сети
8 истинный маркер
8 ответ сети
6 истинный маркер
6 ответ сети
9 истинный маркер
9 ответ сети
4 истинный маркер
4 ответ сети
0 истинный маркер
0 ответ сети
9 истинный маркер
9 ответ сети
1 истинный маркер
1 ответ сети
1 истинный маркер
1 ответ сети
2 истинный маркер
2 ответ сети
4 истинный маркер
4 ответ сети
3 истинный маркер
3 ответ сети
2 истинный маркер
2 ответ сети
7 истинный маркер
7 ответ сети
3 истинный маркер
3 ответ сети
8 истинный маркер
8 ответ сети
6 истинн